In [1]:
import pandas as pd
import numpy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer



In [2]:
df = pd.read_csv('train.dat', sep = '\t', names = ['class', 'text'])

In [3]:
df.head()

,class,text
0,4,Catheterization laboratory events and hospital...
1,5,Renal abscess in children. Three cases of rena...
2,2,Hyperplastic polyps seen at sigmoidoscopy are ...
3,5,Subclavian artery to innominate vein fistula a...
4,4,Effect of local inhibition of gamma-aminobutyr...


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14438 entries, 0 to 14437
Data columns (total 2 columns):
class    14438 non-null int64
text     14438 non-null object
dtypes: int64(1), object(1)
memory usage: 225.7+ KB


In [5]:
df.shape

(14438, 2)

In [6]:
df['class'].value_counts()

5    4805
1    3163
4    3051
3    1925
2    1494
Name: class, dtype: int64

In [7]:

count_vectorizer_tf = TfidfVectorizer(norm= 'l1')
counts_tf = count_vectorizer_tf.fit_transform(df['text'].values)
counts_tf.shape


(14438, 38060)

In [8]:
counts_tf.dtype

dtype('float64')

In [9]:
df_test = pd.read_csv('test.dat', sep = '\n', names = ['text'])

In [10]:
df_test.head()

,text
0,Excision of limbal dermoids. We reviewed the c...
1,Bell's palsy. A diagnosis of exclusion. In cas...
2,Retained endobronchial foreign body removal fa...
3,Recurrent buccal space abscesses: a complicati...
4,Intracranial fibromatosis. Fibromatoses are un...


In [11]:
from sklearn.svm import LinearSVC
targets = df['class'].values
clf = LinearSVC(penalty = 'l1', dual = False, C = 3.1 ,class_weight = 'balanced', random_state = 5)
clf.fit(counts_tf, targets)

LinearSVC(C=3.1, class_weight='balanced', dual=False, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l1', random_state=5, tol=0.0001,
     verbose=0)

In [12]:
counts_test = count_vectorizer_tf.transform(df_test['text'].values)

In [13]:
clf.fit(counts_tf, targets)
predictions = clf.predict(counts_test)

In [14]:
print predictions

[5 5 5 ..., 1 3 5]


In [15]:
prediction=pd.DataFrame(predictions)
prediction.to_csv('result.dat', header = False, index = False)

In [16]:
from sklearn.cross_validation import KFold
from sklearn.metrics import confusion_matrix, f1_score

C:\Users\arpit\Anaconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [17]:
k_fold = KFold(n=len(df), n_folds=6)
scores = []
confusion = numpy.array([[0, 0], [0, 0]])

In [18]:
for train_indices, test_indices in k_fold:
    train_text = df.iloc[train_indices]['text'].values
    train_y = df.iloc[train_indices]['class'].values
    test_text = df.iloc[test_indices]['text'].values
    test_y = df.iloc[test_indices]['class'].values
    
    #counts1 = count_vectorizer.fit_transform(train_text)
    #counts2 = count_vectorizer.transform(test_text)
    
    #counts1_tf = count_vectorizer_tf.fit_transform(train_text)
    #counts2_tf = count_vectorizer_tf.transform(test_text)
    
    counts1_tf1 = count_vectorizer_tf.fit_transform(train_text)
    counts2_tf2 = count_vectorizer_tf.transform(test_text)
    
    clf.fit(counts1_tf1, train_y)
    pre = clf.predict(counts2_tf2)
    
    #classifier1.fit(counts1_tf, train_y)
    #pre_tf = classifier1.predict(counts2_tf)
    #confusion += confusion_matrix(test_y, pre)
    
    #score = f1_score(test_y, pre, average = 'weighted')
    #scores.append(score)

    #score_tf = fl_score(test_y, pre_tf, average = 'weighted')
   # scores_tf.append(score_tf)

In [19]:
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(test_y, pre, average = 'weighted')

(0.62206446178995101, 0.62011637572734835, 0.61024861563561539, None)